In [ ]:

import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, Input,Layer
from scipy.fftpack import dct
# Configurazione per la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limita la memoria GPU allocata (opzionale)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Permette allocazione dinamica della memoria
        print(f"{len(gpus)} GPU(s) disponibile/i.")
    except RuntimeError as e:
        print(e)
else:
    print("Nessuna GPU disponibile, verrà usata la CPU.")

# Carica il dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

def apply_dct(data):
    # Esegui la DCT bidimensionale sull'immagine
    # La `type=2` rappresenta la DCT-II, che è la più comune
    return dct(dct(data, axis=0, norm='ortho'), axis=1, norm='ortho')

X_train_dct = np.array([apply_dct(img) for img in X_train])  # Applica la DCT alle immagini di training
X_test_dct = np.array([apply_dct(img) for img in X_test])  # Applica la DCT alle immagini di test


print("Forma di X_train_dct:", X_train_dct.shape)
print("Forma di X_test_dct:", X_test_dct.shape)

I0000 00:00:1734615879.969977    4031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734615880.215781    4031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734615880.215970    4031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


1 GPU(s) disponibile/i.
Forma di X_train_dct: (60000, 28, 28)
Forma di X_test_dct: (10000, 28, 28)


In [4]:
filtri = np.random.rand(32, 3, 3)

filtri_dct = np.array([apply_dct(img) for img in filtri])  # Applica la DCT alle immagini di training


# Stampa i filtri generati
for i, filtro in enumerate(filtri_dct):
    print(f"Filtro {i+1}:\n{filtro}\n")

Filtro 1:
[[ 1.27426319 -0.32747929  0.24036387]
 [-0.25209554  0.47350744  0.1133368 ]
 [ 0.14519337  0.2925466  -0.01371382]]

Filtro 2:
[[ 1.32167384 -0.05474537 -0.27590598]
 [-0.08464085  0.24898129  0.23818233]
 [ 0.14439755 -0.11880832  0.23966275]]

Filtro 3:
[[ 1.00743892 -0.36226673  0.04060911]
 [ 0.03506639  0.36536688 -0.39191311]
 [-0.26425862  0.05373382 -0.20884649]]

Filtro 4:
[[ 1.37887417 -0.32237218  0.37026075]
 [-0.37246559 -0.22791245  0.05884172]
 [ 0.51335972 -0.22710401  0.13582984]]

Filtro 5:
[[ 1.77420901  0.26023286 -0.25615593]
 [-0.08950105 -0.1203026  -0.05873209]
 [ 0.62191853 -0.23614317 -0.23455555]]

Filtro 6:
[[ 1.61153631 -0.29450716  0.1319067 ]
 [ 0.21727679  0.55528869  0.00862945]
 [ 0.18517728  0.05285956  0.22463321]]

Filtro 7:
[[ 1.66418294 -0.00256606 -0.14292158]
 [ 0.6760698  -0.04005696  0.13704973]
 [-0.29192693 -0.00438004  0.17263226]]

Filtro 8:
[[ 1.76764448 -0.00381654  0.18848864]
 [-0.17600702 -0.30869354  0.26108786]
 [ 0.4926

In [5]:
#rimuovi tutte le colonne apparte prime 3 in tutte le immagini di training e test
#X_train_dct = X_train_dct[:, :, :3]
#X_test_dct = X_test_dct[:, :, :3]



#printa le shape

print("Forma di X_train_dct:", X_train_dct.shape)
print("Forma di X_test_dct:", X_test_dct.shape)


Forma di X_train_dct: (60000, 28, 28)
Forma di X_test_dct: (10000, 28, 28)


In [6]:
class IDCTLayer(Layer):
    def __init__(self, **kwargs):
        super(IDCTLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # IDCT lungo l'asse delle righe e poi delle colonne
        def idct_2d(tensor):
            # Applica la IDCT lungo le righe (asse=-1)
            x = tf.signal.idct(tensor, type=2, norm='ortho', axis=-1)
            # Trasponi per applicare la IDCT lungo le colonne
            x = tf.transpose(x, perm=[0, 1, 3, 2])  # Scambia -2 (righe) con -1 (colonne)
            x = tf.signal.idct(x, type=2, norm='ortho', axis=-1)
            # Ritorna alla disposizione originale
            x = tf.transpose(x, perm=[0, 1, 3, 2])  # Ritorna alla forma originale
            return x

        # Applica la IDCT batch-wise e canale per canale
        return idct_2d(inputs)

In [7]:
model = Sequential([
    Input(shape=(28, 3)),  
    Flatten(),             
    Dense(228, activation='relu'),
    Dense(576, activation='relu'),
    Dense(1152, activation='relu'),
    Reshape((3, 3, 128)),
    IDCTLayer(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

I0000 00:00:1734615882.105594    4031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734615882.106098    4031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734615882.106560    4031 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1734615882.214961    4031 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [8]:
# Compila il modello
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Addestra il modello
history = model.fit(X_train_dct, y_train, epochs=30, batch_size=64, validation_split=0.2)

# Valuta il modello
test_loss, test_acc = model.evaluate(X_test_dct, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")


Epoch 1/30


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 84, but received input with shape (64, 784)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(64, 28, 28), dtype=float32)
  • training=True
  • mask=None

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape

# Configurazione per la GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Limita la memoria GPU allocata (opzionale)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)  # Permette allocazione dinamica della memoria
        print(f"{len(gpus)} GPU(s) disponibile/i.")
    except RuntimeError as e:
        print(e)
else:
    print("Nessuna GPU disponibile, verrà usata la CPU.")

# Carica il dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()



# Crea il modello
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])


# Compila il modello
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Addestra il modello
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Valuta il modello
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")


1 GPU(s) disponibile/i.
Epoch 1/10
319/750 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7094 - loss: 10.2375

KeyboardInterrupt: 

In [38]:


class MatrixMultiplicationLayer(Layer):
    def __init__(self, matrix):
        super(MatrixMultiplicationLayer, self).__init__()
        # La matrice di moltiplicazione è un parametro che viene passato al layer
        self.matrix = tf.constant(matrix, dtype=tf.float32)

    def call(self, inputs):
        print("input: ",inputs.shape)
        if(len(inputs.shape) == 4):
            inputs_reshaped = tf.squeeze(inputs, axis=-1)  # Shape finale: (batch_size, 28, 3)
            print("input reshaped",inputs_reshaped.shape)
        print("matrix shape",self.matrix.shape)
        prova= tf.matmul(inputs_reshaped, self.matrix)
        matrix_transposed = tf.transpose(self.matrix, perm=[2, 1, 0])  # Trasposizione: (32, 3, 3)
        print("prova shape ",prova.shape)
        return prova

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_dct = np.array([apply_dct(img) for img in X_train])  # Applica la DCT alle immagini di training
X_test_dct = np.array([apply_dct(img) for img in X_test])  # Applica la DCT alle immagini di test
X_train_dct = X_train_dct[:, :28, :3]  # Mantieni le prime 28 righe e 3 colonne
X_test_dct = X_test_dct[:, :28, :3]    # Mantieni le prime 28 righe e 3 colonne

# Aggiungi la dimensione per il canale (1 canale per immagine)
X_train_dct = np.expand_dims(X_train_dct, axis=-1)  # Forma finale (num_immagini, 28, 3, 1)
X_test_dct = np.expand_dims(X_test_dct, axis=-1)    # Forma finale (num_immagini, 28, 3, 1)

print(X_train_dct[0].shape)

print("Forma di X_train_dct:", X_train_dct.shape)
print("Forma di X_test_dct:", X_test_dct.shape)

# Crea una matrice di moltiplicazione a mano
filtri1 = np.random.rand(3, 3, 32)
filtri_dct1 = np.array([apply_dct(img) for img in filtri1])  # Applica la DCT alle immagini di training
filtri2 = np.random.rand(3, 3, 64)
filtri_dct2 = np.array([apply_dct(img) for img in filtri2])  # Applica la DCT alle immagini di training
filtri3 = np.random.rand(3, 3, 128)
filtri_dct3 = np.array([apply_dct(img) for img in filtri3])  # Applica la DCT alle immagini di training

# Esempio di utilizzo nel modello:
model = tf.keras.Sequential([
    Input(shape=(28, 3, 1)),  # Ingresso con forma (28, 3, 1)
    MatrixMultiplicationLayer(matrix=filtri_dct1),  # Usa la matrice definita
    MatrixMultiplicationLayer(matrix=filtri_dct2),  # Usa la matrice definita
    MatrixMultiplicationLayer(matrix=filtri_dct3),  # Usa la matrice definita
    Reshape((28, 32)),  # Reshape per ottenere un output di forma (28, 32)
    Dense(1)  # Aggiungi un altro layer denso per l'output
])

(28, 3, 1)
Forma di X_train_dct: (60000, 28, 3, 1)
Forma di X_test_dct: (10000, 28, 3, 1)
input:  (None, 28, 3, 1)
input reshaped (None, 28, 3)
matrix shape (3, 3, 32)
prova shape  (3, 28, 32)
input:  (3, 28, 32)
matrix shape (3, 3, 64)


UnboundLocalError: Exception encountered when calling MatrixMultiplicationLayer.call().

[1mCould not automatically infer the output shape / dtype of 'matrix_multiplication_layer_56' (of type MatrixMultiplicationLayer). Either the `MatrixMultiplicationLayer.call()` method is incorrect, or you need to implement the `MatrixMultiplicationLayer.compute_output_spec() / compute_output_shape()` method. Error encountered:

cannot access local variable 'inputs_reshaped' where it is not associated with a value[0m

Arguments received by MatrixMultiplicationLayer.call():
  • args=('<KerasTensor shape=(3, 28, 32), dtype=float32, sparse=False, name=keras_tensor_49>',)
  • kwargs=<class 'inspect._empty'>